# Pre-Trained tokenizer 사용하기   
BPE Tokenizer는 구글의 Sentencepiece를 많이 사용한다(Sentencepiece 사용법 : https://github.com/google/sentencepiece)

In [1]:
# 이미 학습된 Tokenizer와 vocabulary 가져오기
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # bert-base-uncased라는 모델을 사용하기 위해 Tokenizer을 일치시키기 위해 미리 학습된 Tokenizer가져옴

print(len(tokenizer.vocab))

30522


In [2]:
# 토크니저가 문장을 Tokenization하는 방법
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentence = 'My dog is cute. He likes playing'
print(tokenizer.tokenize(sentence))                                       # tokenization 결과 보기

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
print(len(tokenizer.vocab))
print(tokenizer.tokenize(sentence))                                       # 

sentence = '나는 책상 위에 사과를 먹었다. 알고 보니 그 사과는 Jason 것이었다. 그래서 Jason에게 사과를 했다'
print(tokenizer.tokenize(sentence))

['my', 'dog', 'is', 'cute', '.', 'he', 'likes', 'playing']
105879
['my', 'dog', 'is', 'cut', '##e', '.', 'he', 'likes', 'playing']
['나는', 'ᄎ', '##ᅢᆨ', '##상', '위에', 'ᄉ', '##ᅡ', '##과', '##를', 'ᄆ', '##ᅥ', '##ᆨ', '##었다', '.', '알', '##고', 'ᄇ', '##ᅩ', '##니', '그', 'ᄉ', '##ᅡ', '##과', '##는', 'jason', '것이', '##었다', '.', '그', '##래', '##서', 'jason', '##에게', 'ᄉ', '##ᅡ', '##과', '##를', '했다']


In [12]:
# 데이터 전처리 및 Pre-Trained Embedding Vector를 이용한 Vocabulary 생성
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

# Data Setting
TEXT = data.Field(batch_first = True,
                 fix_length = 500,       # Sentence 길이 제한
                 tokenize=str.split,     # Tokenize를 설정하는 옵션 띄어쓰기 기반으로 설정
                 pad_first = True,       # fix_length 보다 짧은 경우 패딩을 앞에하라
                 pad_token='[PAD]',      # 패딩에 대한 특수 토큰 설정
                 unk_token='[UNK]')      # 사전에 없는 토큰의 경우 특수 토큰

LABEL = data.LabelField(dtype=torch.float)  # 가져올 데이터에 대한 타입 설정
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [13]:
# 불러온 데이터 확인

# 데이터 길이
print(f'Train Data Length: {len(train_data.examples)}')
print(f'Test Data Length: {len(test_data.examples)}')

# 데이터 필드
print(train_data.fields)

# 데이터 샘플
print('--- Data Sample ---')
print('Input : ')
print(' '.join(vars(train_data.examples[1])['text']), '\\n')
print('Label : ')
print(vars(train_data.examples[1])['label'])

Train Data Length: 25000
Test Data Length: 25000
{'text': <torchtext.legacy.data.field.Field object at 0x00000251431D3E50>, 'label': <torchtext.legacy.data.field.LabelField object at 0x00000251431D3EB0>}
--- Data Sample ---
Input : 
Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson

In [14]:
# 데이터 클리닝 작업하기
import re

def PreProcessingText(input_sentence):
    input_sentence = input_sentence.lower()
    input_sentence = re.sub('<[^>]*>', repl=' ', string = input_sentence)  # <br /> 처리
    input_sentence = re.sub('[!"#$%&\\()*+, -./:;<=>?@[\\\\]^_`{|}~"]', repl= ' ', string = input_sentence)  # 특수 문자 처리
    input_sentence = re.sub('\\s+', repl=' ', string = input_sentence)  # 연속된 띄어쓰기 처리
    if input_sentence:
        return input_sentence
for example in train_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()
for example in test_data.examples:
    vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()

In [15]:
# 주어진 데이터로 토큰 vocabulary를 만들기
TEXT.build_vocab(train_data,
                min_freq = 2,                # Vocab에 해당하는 토큰에 최소한으로 등장하는 횟수에 제한을 둔다
                max_size = None,             # 토큰의 최소 등장 회수로 vocab 의 사이즈를 조절하는 것 외에 전체 Vocab 자체에 제한을 둔다
                vectors = 'glove.6B.300d')   # 사전 학습 Vector를 가져와 Vocab에 세팅하는 옵션. 원하는 임베딩을 string 형태로 설정
LABEL.build_vocab(train_data)

.vector_cache\glove.6B.zip: 862MB [12:30, 1.15MB/s]                                                                    
100%|███████████████████████████████████████████████████████████████████████▉| 399999/400000 [01:03<00:00, 6291.90it/s]


In [17]:
# 배치 데이터 만들기
import random

train_data, valid_data = train_data.split(random_state = random.seed(0), split_ratio=0.8)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(datasets=(train_data, valid_data, test_data), batch_size=30, device=device)

## Models
NLP 모델로는 RNN, LSTM, Bi-RNNs, GRUs 등이 있다.

In [21]:
import torch.nn as nn
class SentenceClassification(nn.Module):
    def __init__(self, **model_config):
        super(sentenceClassification, self).__init__()
        
        if model_config['emb_type'] == 'glove' or 'fasttext':
            self.emb = nn.Embedding(model_config['vocab_size'],   # 임베딩 설정, num_embedding  : Vocab Size
                                   model_config['emb_dim'],       # embedding_dim : 원하는 임베딩 Dimension 설정
                                   _weight = TEXT.vocab.vectors)  # 
        else:
            self.emb = nn.Embedding(model_config['vocab_size'],
                                   model_config['emb_dim'])
        self.bidirectional = model_config['bidirectional']
        self.num_direction = 2 if model_config['bidirectional'] else 1
        self.model_type = model_config['model_type']
        
        self.RNN = nn.RNN(input_size = model_config['emb_dim'],         # 입력받을 데이터 크기
                         hidden_size = model_config['hidden_dim'],      # Hidden Layer의 Dimension 설정
                         dropout = model_config['dropout'],             # 드롭 아웃의 확률 설정
                         bidirectional = model_config['bidirectional'], # 양방향 모델 사용시 설정
                         batch_first = model_config['batch_first'])     # Data의 제일 처음 Axis에 배치 사이즈가 오도록 설정
        self.LSTM = nn.LSTM(input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectinal = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        self.GRU = nn.GRU(input_size = model_config['emb_dim'],
                         hidden_size = model_config['hidden_dim'],
                         dropout = model_config['dropout'],
                         bidirectinal = model_config['bidirectional'],
                         batch_first = model_config['batch_first'])
        self.fc = nn.Linear(model_config['hidden_dim'] * self.num_direction, model_config['output_dim']) 
        self.drop = nn.Dropout(model_config['dropout'])
    def forward(self, x):
        emb = self.emb(x)
        if self.model_type == 'RNN':
            output, hidden = self.RNN(emb)
        elif self.model_type == 'LSTM':
            output, (hidden, cell) = self.LSTM(emb)
        elif self.model_type == 'GRU':
            output, hidden = self.GRU(emb)
        else:
            raise NameError('Select model_type in [RNN, LSTM, GRU]')
        last_output = output[:,-1,:]
        return self.fc(self.drop(last_output))

In [24]:
model_config.update(dict(batch_first = True,
                        model_type = 'RNN',
                        bidirectional = True,
                        hidden_dim = 128,
                        output_dim = 1,
                        dropout = 0))
model = SentenceClassification(**model_config)
loss_fn = nn.BCEWithLogitsLoss()

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum()/len(correct)
    return acc

predictions = model.forward(sample_for_check.text).squeeze()
loss = loss_fn(predictions, sample_for_check.label)
acc = binary_accuracy(predictions, sample_for_chec.label)

print(predictions)
print(loss, acc)

NameError: name 'model_config' is not defined